# Mirrored Strategy: Basic


## Imports

In [1]:
# Import TensorFlow and TensorFlow Datasets

import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

Load the MNIST dataset and split it into training and test chunks

In [2]:
# Load the dataset we'll use for this lab
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True, data_dir='./data')

mnist_train, mnist_test = datasets['train'], datasets['test']

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dataset mnist downloaded and prepared to ./data/mnist/3.0.1. Subsequent calls will reuse this data.


Next, we define `strategy` using the `MirroredStrategy()` class. Print to see the number of devices available.


In [3]:
# Define the strategy to use and print the number of devices found
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Number of devices: 1


Next, you create your training and test examples, define your batch size and also define `BATCH_SIZE_PER_REPLICA` which is the distribution you are making for each available device.

In [4]:
# Get the number of examples in the train and test sets
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
# Use for Mirrored Strategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
# Use for No Strategy
# BATCH_SIZE = BATCH_SIZE_PER_REPLICA * 1

A mapping function which normalizes our images

In [5]:
# Function for normalizing the image
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

Next we create training and evaluation datesets in the batch size you want by shuffling through your buffer size.

In [6]:
# Set up the train and eval data set
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

For our model to follow the strategy, we define model within the strategy's scope.


The important thing to notice and compare is the time taken for each epoch to complete.

In [7]:
# Use for Mirrored Strategy -- comment out `with strategy.scope():` and deindent for no strategy
with strategy.scope():
  model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

In [8]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [9]:
model.fit(train_dataset, epochs=12)

Epoch 1/12
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


938/938 [==============================] - 25s 8ms/step - loss: 0.2024 - accuracy: 0.9412
Epoch 2/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0666 - accuracy: 0.9803
Epoch 3/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0469 - accuracy: 0.9858
Epoch 4/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0344 - accuracy: 0.9896
Epoch 5/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0274 - accuracy: 0.9912
Epoch 6/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0210 - accuracy: 0.9937
Epoch 7/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0161 - accuracy: 0.9949
Epoch 8/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0109 - accuracy: 0.9970
Epoch 9/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0094 - accuracy: 0.9971
Epoch 10/12
938/938 [==============================] - 3s 3ms/step - loss: 0.0071 - accuracy: 0.9977
Epoch 11/

Let's compare the speed with simple non-distributed training.

In [10]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model.fit(train_dataset, epochs=12)

Epoch 1/12
938/938 [==============================] - 3s 2ms/step - loss: 0.2049 - accuracy: 0.9398
Epoch 2/12
938/938 [==============================] - 2s 2ms/step - loss: 0.0684 - accuracy: 0.9798
Epoch 3/12
938/938 [==============================] - 2s 2ms/step - loss: 0.0484 - accuracy: 0.9851
Epoch 4/12
938/938 [==============================] - 2s 2ms/step - loss: 0.0362 - accuracy: 0.9891
Epoch 5/12
938/938 [==============================] - 2s 2ms/step - loss: 0.0280 - accuracy: 0.9914
Epoch 6/12
938/938 [==============================] - 2s 2ms/step - loss: 0.0214 - accuracy: 0.9936
Epoch 7/12
938/938 [==============================] - 2s 2ms/step - loss: 0.0170 - accuracy: 0.9947
Epoch 8/12
938/938 [==============================] - 2s 2ms/step - loss: 0.0124 - accuracy: 0.9961
Epoch 9/12
938/938 [==============================] - 2s 2ms/step - loss: 0.0101 - accuracy: 0.9970
Epoch 10/12
938/938 [==============================] - 2s 2ms/step - loss: 0.0078 - accuracy: 0.9979

Having just one available GPU it's slower to use this distributed strategy.